## 필요한 라이브러리 로드

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
pd.options.display.float_format = '{:.5f}'.format

금액과 관련된 변수를 지수형태로 나타나지 않게 위한 작업을 시행한다.

## csv파일 데이터 로드 및 변수 선택

In [4]:
df_gg = pd.read_csv(r'C:\Users\moon_\OneDrive - konkuk.ac.kr\바탕 화면\조달청 공모전\data\FILE/물품입찰공고.csv') #물품입찰공고
df_cf = pd.read_csv(r'C:\Users\moon_\OneDrive - konkuk.ac.kr\바탕 화면\조달청 공모전\data\FILE/물품입찰분류별진행내역.csv') #물품입찰분류별진행내역
df_tc1 = pd.read_csv(r'C:\Users\moon_\OneDrive - konkuk.ac.kr\바탕 화면\조달청 공모전\data\FILE/투찰업체1.csv') #투찰업체1
df_tc2 = pd.read_csv(r'C:\Users\moon_\OneDrive - konkuk.ac.kr\바탕 화면\조달청 공모전\data\FILE/투찰업체2.csv') #투찰업체2
df_tc3 = pd.read_csv(r'C:\Users\moon_\OneDrive - konkuk.ac.kr\바탕 화면\조달청 공모전\data\FILE/투찰업체3.csv') #투찰업체3
df_tc4 = pd.read_csv(r'C:\Users\moon_\OneDrive - konkuk.ac.kr\바탕 화면\조달청 공모전\data\FILE/투찰업체4.csv') #투찰업체4

C:\Users\moon_\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df_tc = pd.concat([df_tc1, df_tc2, df_tc3, df_tc4])

물품의 경우, 투찰 업체의 내역이 상당히 많으므로 4번을 나누어 다운로드 하였으므로 하나로 합치는 과정을 가짐.

In [6]:
df_gg_features = df_gg[['입찰공고번호', '공동도급협정서접수방식']]

df_cf_features = df_cf[['입찰공고번호', '합계수량', '낙찰자결정여부', '낙찰업체투찰률', '낙찰업체투찰금액', '낙찰자결정적용법규']]

df_tc_features = df_tc[['등록유형', '조달구분', '입찰공고번호', '품명내용', '긴급공고여부', '추정가격', '배정예산',
                        '입찰방식', '기초금액', '예정가격', '낙찰여부', '낙찰자결정방법', '낙찰하한율', '참가수']]

모델링 전 전처리 과정에서 실제로 null값을 가진 값을 drop, data의 value 대체 등 조건을 걸어 전처리를 진행한 변수들만 선택하였다.

In [7]:
df_2 = pd.merge(df_tc_features, df_cf_features, how='inner', on='입찰공고번호')

In [8]:
df = pd.merge(df_2, df_gg_features, how='inner', on='입찰공고번호')

merge를 통하여 세 가지 csv파일에서 변수를 선택한 후 입찰공고번호를 기준으로 inner join하여 하나의 데이터프레임으로 합병한다.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 303597 entries, 0 to 303596
Data columns (total 20 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   등록유형         303597 non-null  object 
 1   조달구분         303597 non-null  object 
 2   입찰공고번호       303597 non-null  int64  
 3   품명내용         303596 non-null  object 
 4   긴급공고여부       289037 non-null  object 
 5   추정가격         302274 non-null  float64
 6   배정예산         301334 non-null  float64
 7   입찰방식         303597 non-null  object 
 8   기초금액         182167 non-null  float64
 9   예정가격         302376 non-null  float64
 10  낙찰여부         303597 non-null  object 
 11  낙찰자결정방법      302571 non-null  object 
 12  낙찰하한율        182781 non-null  float64
 13  참가수          303597 non-null  int64  
 14  합계수량         303597 non-null  float64
 15  낙찰자결정여부      303597 non-null  object 
 16  낙찰업체투찰률      246724 non-null  float64
 17  낙찰업체투찰금액     246724 non-null  float64
 18  낙찰자결정적용법규    271019 non-

## data 전처리

In [10]:
df = df[df['등록유형'] == '나라장터(G2B)']
df = df.drop(['등록유형'], axis=1)

등록유형을 나라장터로 한정하고, 등록유형을 drop한다.

In [11]:
df = df[df['낙찰여부'] == 'Y']
df = df.drop(['낙찰여부'], axis=1)

유찰이 된 data는 가격 예측에 도움이 되지 않는다. 낙찰이 이루어진 이전 입찰 data를 통해 예측을 진행해야 하므로, 낙찰이 된 data로 한정한다.

In [12]:
df = df.dropna(subset=['품명내용'])
df = df.dropna(subset=['긴급공고여부'])
df = df.dropna(subset=['추정가격'])
df = df.dropna(subset=['배정예산'])

품명내용, 긴급공고여부, 추정가격, 배정예산이 na인 값은 대체가 불가능하므로 drop한다.

In [13]:
df = df[df['입찰방식'].str.contains('전자')]
df = df.drop(['입찰방식'], axis=1)

df.head(3)

,조달구분,입찰공고번호,품명내용,긴급공고여부,추정가격,배정예산,기초금액,예정가격,낙찰자결정방법,낙찰하한율,참가수,합계수량,낙찰자결정여부,낙찰업체투찰률,낙찰업체투찰금액,낙찰자결정적용법규,공동도급협정서접수방식
0,자체조달,20211020431,교육용또는직업용도서[55101509],N,17389840.00000,17389840.00000,17389840.00000,17491500.00000,제한적최저가(낙찰하한율),90.00000,9,1336.00000,Y,90.08200,15756760.00000,지방계약법,일반(중앙)
6,자체조달,20211027248,구내교환장비[43222805],Y,82150000.00000,90365000.00000,90365000.00000,89554000.00000,적격심사(추정가격 고시금액미만),84.24500,26,1.00000,Y,84.37700,75563000.00000,지방계약법,일반(중앙)
7,자체조달,20211028370,실험용플라스크[41121804],N,48992800.00000,53892080.00000,53892080.00000,53986325.00000,"적격심사(추정가격 고시금액미만 제조입찰,고시금액미만 구매입찰)",84.24500,8,1.00000,Y,92.52500,49950890.00000,국가계약법,일반(중앙)


전자입찰과 관련된 입찰방식으로 추린다.

In [14]:
df['낙찰하한율'].isna().sum() #낙찰하한율 대체 전

105666

null값의 대체가 가능한 낙찰하한율은 대체한다.

In [15]:
df[(df['낙찰자결정방법'].isna()==True) & (df['낙찰하한율'].isna()==False)][['낙찰자결정방법', '낙찰하한율']]

,낙찰자결정방법,낙찰하한율
135,NaN,87.94500
142,NaN,87.94500
332,NaN,88.00000
546,NaN,87.99500
799,NaN,87.99500
...,...,...
301754,NaN,87.99500
302140,NaN,88.00000
302203,NaN,88.00000
302394,NaN,88.00000


낙찰자결정방법을 통해 낙찰하한율을 대체할 예정이다. 그러나 낙찰자결정방법이 결측치인 데이터를 모두 제거하면, 낙찰자결정방법은 결측이지만 낙찰하한율은 결측이 아닌 데이터까지 제거하게 되므로 낙찰하한율이 있는 경우에는 낙찰자결정방법을 drop하지 않는다.

In [16]:
df['낙찰자결정방법'].fillna('알수없음', inplace=True)

In [17]:
df.loc[df['낙찰자결정방법'].str.contains('10억이상'), '낙찰하한율'] = 80.495
df.loc[df['낙찰자결정방법'].str.contains('10억원이상'), '낙찰하한율'] = 80.495

df.loc[df['낙찰자결정방법'].str.contains('10억원미만'), '낙찰하한율'] = 80.495
df.loc[df['낙찰자결정방법'].str.contains('10억미만'), '낙찰하한율'] = 80.495

df.loc[df['낙찰자결정방법'].str.contains('고시금액미만'), '낙찰하한율'] = 84.245
df.loc[df['낙찰자결정방법'].str.contains('고시금액 미만'), '낙찰하한율'] = 84.245

df.loc[df['낙찰자결정방법'].str.contains('간행물'), '낙찰하한율'] = 89.995

낙찰하한율을 대체할 수 있는 정보가 한정적이여서 많이 대체하지는 못했지만, 정보가 업데이트 된다면 모델을 업데이트 할 필요가 있어보임.

In [18]:
df['낙찰하한율'].isna().sum() #낙찰하한율 대체 후

105265

낙찰하한율의 null값의 개수가 감소하였음.

In [19]:
df = df.dropna(subset=['낙찰하한율'])

낙찰하한율의 나머지 null값은 처리가 불가능하므로 drop한다.

In [20]:
df = df.dropna(subset=['기초금액'])
df = df.dropna(subset=['낙찰업체투찰금액'])

금액과 관련된 변수는 대체 시 data상의 오류가 발생할 가능성이 있으므로 null값을 drop한다.

금액과 관련된 변수는 null값이 없어야 하며 대체도 불가능하다.

In [21]:
df = df.dropna(subset=['합계수량'])
df = df.dropna(subset=['낙찰자결정적용법규'])
df = df.dropna(subset=['공동도급협정서접수방식'])
df = df.dropna(subset=['참가수'])
df = df[df['조달구분']=='자체조달']

df.head(3)

,조달구분,입찰공고번호,품명내용,긴급공고여부,추정가격,배정예산,기초금액,예정가격,낙찰자결정방법,낙찰하한율,참가수,합계수량,낙찰자결정여부,낙찰업체투찰률,낙찰업체투찰금액,낙찰자결정적용법규,공동도급협정서접수방식
0,자체조달,20211020431,교육용또는직업용도서[55101509],N,17389840.00000,17389840.00000,17389840.00000,17491500.00000,제한적최저가(낙찰하한율),90.00000,9,1336.00000,Y,90.08200,15756760.00000,지방계약법,일반(중앙)
6,자체조달,20211027248,구내교환장비[43222805],Y,82150000.00000,90365000.00000,90365000.00000,89554000.00000,적격심사(추정가격 고시금액미만),84.24500,26,1.00000,Y,84.37700,75563000.00000,지방계약법,일반(중앙)
7,자체조달,20211028370,실험용플라스크[41121804],N,48992800.00000,53892080.00000,53892080.00000,53986325.00000,"적격심사(추정가격 고시금액미만 제조입찰,고시금액미만 구매입찰)",84.24500,8,1.00000,Y,92.52500,49950890.00000,국가계약법,일반(중앙)


합계수량과 낙찰자 결정 적용 법규, 공동도급 협정서 접수 방식, 참가수를 다른 값으로 대체한다면 data상의 오류가 발생할 가능성이 있다. 따라서 drop하고, EDA 당시 중앙조달의 data 수가 매우 적어 가격 예측에 도움이 되지 못할 것 같아 자체조달로 한정하였으므로 자체조달로 한정한다.

In [22]:
df = df[df['낙찰업체투찰금액'] != 0]

낙찰업체투찰금액이 0원인 값은 Null로 간주하여 0원을 제거한다.

In [23]:
Q1 = np.percentile(df['낙찰업체투찰금액'].values, 25)
Q3 = np.percentile(df['낙찰업체투찰금액'].values, 75)

IQR = Q3-Q1

df = df[(df['낙찰업체투찰금액'] >= (Q1 - (1.5*IQR))) & (df['낙찰업체투찰금액'] <= (Q3 + (1.5*IQR)))]

낙찰업체투찰금액의 편차가 매우 크므로, 이상치를 제거한다.

In [24]:
df = df[df['낙찰하한율'] >= 79]

df.head(3)

,조달구분,입찰공고번호,품명내용,긴급공고여부,추정가격,배정예산,기초금액,예정가격,낙찰자결정방법,낙찰하한율,참가수,합계수량,낙찰자결정여부,낙찰업체투찰률,낙찰업체투찰금액,낙찰자결정적용법규,공동도급협정서접수방식
0,자체조달,20211020431,교육용또는직업용도서[55101509],N,17389840.00000,17389840.00000,17389840.00000,17491500.00000,제한적최저가(낙찰하한율),90.00000,9,1336.00000,Y,90.08200,15756760.00000,지방계약법,일반(중앙)
17,자체조달,20211032984,차양[30151901],N,47745000.00000,52519500.00000,100000.00000,99200.00000,제한적최저가(낙찰하한율),88.00000,4,1.00000,Y,88.32600,87620.00000,지방계약법,일반(중앙)
40,자체조달,20211034432,따로분류되지않고치료를주목적으로하지않는의약품[51320401],N,26175454.00000,28793000.00000,28793000.00000,29055075.00000,제한적최저가(낙찰하한율),88.00000,4,1.00000,Y,95.93300,27873600.00000,지방계약법,일반(중앙)


낙찰하한율이 79% 이하인 것은 이상치로 판단하고 79 이상인 것들로만 간추린다.

In [25]:
df.loc[df['낙찰자결정방법'].str.contains('적격심사'), '낙찰자결정방법'] = '적격심사낙찰제'
df.loc[df['낙찰자결정방법'].str.contains('최저가'), '낙찰자결정방법'] = '최저가낙찰제'
df.loc[df['낙찰자결정방법'].str.contains('계약이행능력심사'), '낙찰자결정방법'] = '계약이행능력심사'
df.loc[df['낙찰자결정방법'].str.contains('희망수량경쟁'), '낙찰자결정방법'] = '희망수량경쟁'

df.낙찰자결정방법.value_counts()

적격심사낙찰제     96167
최저가낙찰제      23287
계약이행능력심사      300
알수없음           18
희망수량경쟁         15
Name: 낙찰자결정방법, dtype: int64

낙찰자 결정방법의 명칭을 통일한다.

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119787 entries, 0 to 303593
Data columns (total 17 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   조달구분         119787 non-null  object 
 1   입찰공고번호       119787 non-null  int64  
 2   품명내용         119787 non-null  object 
 3   긴급공고여부       119787 non-null  object 
 4   추정가격         119787 non-null  float64
 5   배정예산         119787 non-null  float64
 6   기초금액         119787 non-null  float64
 7   예정가격         119787 non-null  float64
 8   낙찰자결정방법      119787 non-null  object 
 9   낙찰하한율        119787 non-null  float64
 10  참가수          119787 non-null  int64  
 11  합계수량         119787 non-null  float64
 12  낙찰자결정여부      119787 non-null  object 
 13  낙찰업체투찰률      119787 non-null  float64
 14  낙찰업체투찰금액     119787 non-null  float64
 15  낙찰자결정적용법규    119787 non-null  object 
 16  공동도급협정서접수방식  119787 non-null  object 
dtypes: float64(8), int64(2), object(7)
memory usage: 16.5+ MB


data의 전처리는 진행되었으므로, 필요한 변수를 제외하고 전부 삭제한다.

In [27]:
df = df[['낙찰자결정방법', '참가수', '합계수량', '기초금액', '배정예산', '낙찰업체투찰금액']]

df.head(3)

,낙찰자결정방법,참가수,합계수량,기초금액,배정예산,낙찰업체투찰금액
0,최저가낙찰제,9,1336.00000,17389840.00000,17389840.00000,15756760.00000
17,최저가낙찰제,4,1.00000,100000.00000,52519500.00000,87620.00000
40,최저가낙찰제,4,1.00000,28793000.00000,28793000.00000,27873600.00000


전처리용 column을 제외하고, 실제 모델링에 필요한 변수들만 간추린다.

In [28]:
df['개당기초금액'] = df['기초금액'] / df['합계수량']
df['개당배정예산'] = df['배정예산'] / df['합계수량']
df['개당낙찰업체투찰금액'] = df['낙찰업체투찰금액'] / df['합계수량']

df.head(3)

,낙찰자결정방법,참가수,합계수량,기초금액,배정예산,낙찰업체투찰금액,개당기초금액,개당배정예산,개당낙찰업체투찰금액
0,최저가낙찰제,9,1336.00000,17389840.00000,17389840.00000,15756760.00000,13016.34731,13016.34731,11793.98204
17,최저가낙찰제,4,1.00000,100000.00000,52519500.00000,87620.00000,100000.00000,52519500.00000,87620.00000
40,최저가낙찰제,4,1.00000,28793000.00000,28793000.00000,27873600.00000,28793000.00000,28793000.00000,27873600.00000


각 data마다 합계수량의 차이가 있으므로, 금액과 관련된 변수를 합계수량으로 나누어 개당 금액을 구한다.

In [29]:
df['log_개당배정예산'] = np.log1p(df['개당배정예산'])
df['log_개당기초금액'] = np.log1p(df['개당기초금액'])
df['log_개당낙찰업체투찰금액'] = np.log1p(df['개당낙찰업체투찰금액'])

df.head(3)

,낙찰자결정방법,참가수,합계수량,기초금액,배정예산,낙찰업체투찰금액,개당기초금액,개당배정예산,개당낙찰업체투찰금액,log_개당배정예산,log_개당기초금액,log_개당낙찰업체투찰금액
0,최저가낙찰제,9,1336.00000,17389840.00000,17389840.00000,15756760.00000,13016.34731,13016.34731,11793.98204,9.47404,9.47404,9.37543
17,최저가낙찰제,4,1.00000,100000.00000,52519500.00000,87620.00000,100000.00000,52519500.00000,87620.00000,17.77670,11.51294,11.38078
40,최저가낙찰제,4,1.00000,28793000.00000,28793000.00000,27873600.00000,28793000.00000,28793000.00000,27873600.00000,17.17564,17.17564,17.14319


이상치 제거를 했음에도 편차가 크므로 log변환을 통하여 편차를 줄여준다.

In [30]:
df_ohe = pd.get_dummies(df)

낙찰자 결정방법이 수치형 변수가 아니므로 더미변수로 만들어 모델링해준다.

In [35]:
df_ohe

,참가수,합계수량,기초금액,배정예산,낙찰업체투찰금액,개당기초금액,개당배정예산,개당낙찰업체투찰금액,log_개당배정예산,log_개당기초금액,log_개당낙찰업체투찰금액,낙찰자결정방법_계약이행능력심사,낙찰자결정방법_알수없음,낙찰자결정방법_적격심사낙찰제,낙찰자결정방법_최저가낙찰제,낙찰자결정방법_희망수량경쟁
0,9,1336.00000,17389840.00000,17389840.00000,15756760.00000,13016.34731,13016.34731,11793.98204,9.47404,9.47404,9.37543,0,0,0,1,0
17,4,1.00000,100000.00000,52519500.00000,87620.00000,100000.00000,52519500.00000,87620.00000,17.77670,11.51294,11.38078,0,0,0,1,0
40,4,1.00000,28793000.00000,28793000.00000,27873600.00000,28793000.00000,28793000.00000,27873600.00000,17.17564,17.17564,17.14319,0,0,0,1,0
44,11,2.00000,28644000.00000,28644000.00000,28050000.00000,14322000.00000,14322000.00000,14025000.00000,16.47731,16.47731,16.45635,0,0,0,1,0
46,4,12.00000,20120000.00000,20120000.00000,19200000.00000,1676666.66667,1676666.66667,1600000.00000,14.33232,14.33232,14.28551,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303554,9,1.00000,18453809.00000,18453809.00000,16269000.00000,18453809.00000,18453809.00000,16269000.00000,16.73078,16.73078,16.60477,0,0,0,1,0
303556,6,1.00000,26335980.00000,26335980.00000,23166000.00000,26335980.00000,26335980.00000,23166000.00000,17.08645,17.08645,16.95820,0,0,0,1,0
303557,5,1.00000,37102600.00000,37102600.00000,33498300.00000,37102600.00000,37102600.00000,33498300.00000,17.42920,17.42920,17.32701,0,0,0,1,0
303558,142,1.00000,19012000.00000,19400000.00000,16758300.00000,19012000.00000,19400000.00000,16758300.00000,16.78078,16.76058,16.63440,0,0,0,1,0


In [32]:
X = df_ohe[['참가수', 'log_개당배정예산', 'log_개당기초금액', '낙찰자결정방법_계약이행능력심사',
       '낙찰자결정방법_적격심사낙찰제', '낙찰자결정방법_최저가낙찰제', '낙찰자결정방법_희망수량경쟁', '낙찰자결정방법_알수없음']]
y = df_ohe[['log_개당낙찰업체투찰금액']]

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=32)

test data와 train data를 분리하여 모델에 fit한다.

In [34]:
print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

X_train shape:  (89840, 8)
X_test shape:  (29947, 8)
y_train shape:  (89840, 1)
y_test shape:  (29947, 1)


## LightGBM Model

In [36]:
lgbm_model = lgb.LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=7525, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [45]:
lgbm_model.fit(X_train, y_train)

lgbm_preds = lgbm_model.predict(X_test)

lgbm_rmse = np.sqrt(mean_squared_error(y_test, lgbm_preds))

print('RMSE: %f' % (lgbm_rmse))
print('RMSLE : ', (np.sqrt(np.mean(np.square(np.log1p(y_test.iloc[:,0]) - np.log1p(lgbm_preds))))))

print('R2 Score : %f' % (r2_score(y_test, lgbm_preds)))
print('adjusted R2 Score : ', 1-(1-r2_score(y_test, lgbm_preds)) * (len(y_test)-1) / (len(y_test) - len(X.columns) - 1))

RMSE: 1.119215
RMSLE :  0.30885706578156213
R2 Score : 0.948997
adjusted R2 Score :  0.9489832707135629


In [46]:
lgbm_check = pd.DataFrame({'log_개당낙찰업체투찰금액' : y_test.iloc[:,0],
              'log_개당낙찰업체투찰금액_lgbm_pre' : lgbm_preds})

lgbm_check['개당낙찰업체투찰금액'] = np.expm1(lgbm_check['log_개당낙찰업체투찰금액'])
lgbm_check['개당낙찰업체투찰금액_lgbm_pre'] = np.expm1(lgbm_check['log_개당낙찰업체투찰금액_lgbm_pre'])

lgbm_check

,log_개당낙찰업체투찰금액,log_개당낙찰업체투찰금액_lgbm_pre,개당낙찰업체투찰금액,개당낙찰업체투찰금액_lgbm_pre
243356,16.74207,16.77198,18663300.00000,19230021.91245
199304,2.88848,2.76059,16.96601,14.80913
193208,3.43420,2.75041,30.00662,14.64908
187364,1.22002,2.76227,2.38726,14.83568
270741,17.00845,16.91744,24360000.00000,22240923.35983
...,...,...,...,...
179468,2.98603,2.79291,18.80689,15.32846
207356,5.09520,2.76771,162.23725,14.92211
158354,4.92559,2.77754,136.77047,15.07940
165553,3.71477,2.79092,40.04929,15.29596


test data와 predict한 값을 비교해본다.

log 변환 처리한 금액을 다시 지수변환으로 데이터를 원상복구해준다.

In [48]:
print('RMSE: %f' % (np.sqrt(mean_squared_error(lgbm_check['개당낙찰업체투찰금액'], lgbm_check['개당낙찰업체투찰금액_lgbm_pre']))))
print('RMSLE : ', (np.sqrt(np.mean(np.square(np.log1p(lgbm_check['개당낙찰업체투찰금액']) - np.log1p(lgbm_check['개당낙찰업체투찰금액_lgbm_pre']))))))

RMSE: 350264.730123
RMSLE :  1.1192152632284698


## XGBoost

In [51]:
xgb_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, 
             monotone_constraints='()', n_estimators=100, n_jobs=-1,
             num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=7525, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='auto',
             validate_parameters=1, verbosity=0)

In [52]:
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)

xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_preds))

print('RMSE: %f' % (xgb_rmse))
print('RMSLE : ', (np.sqrt(np.mean(np.square(np.log1p(y_test.iloc[:,0]) - np.log1p(xgb_preds))))))

print('R2 Score : %f' % (r2_score(y_test, xgb_preds)))
print('adjusted R2 Score : ', 1-(1-r2_score(y_test, xgb_preds)) * (len(y_test)-1) / (len(y_test) - len(X.columns) - 1))

RMSE: 1.122104
RMSLE :  0.30941365922540315
R2 Score : 0.948733
adjusted R2 Score :  0.9487195613994837


In [53]:
xgb_check = pd.DataFrame({'log_개당낙찰업체투찰금액' : y_test.iloc[:,0],
              'log_개당낙찰업체투찰금액_xgb_pre' : xgb_preds})

xgb_check['개당낙찰업체투찰금액'] = np.expm1(xgb_check['log_개당낙찰업체투찰금액'])
xgb_check['개당낙찰업체투찰금액_xgb_pre'] = np.expm1(xgb_check['log_개당낙찰업체투찰금액_xgb_pre'])

xgb_check

,log_개당낙찰업체투찰금액,log_개당낙찰업체투찰금액_xgb_pre,개당낙찰업체투찰금액,개당낙찰업체투찰금액_xgb_pre
243356,16.74207,16.74778,18663300.00000,18770176.00000
199304,2.88848,2.73916,16.96601,14.47404
193208,3.43420,2.74812,30.00662,14.61329
187364,1.22002,2.77183,2.38726,14.98784
270741,17.00845,16.94390,24360000.00000,22837148.00000
...,...,...,...,...
179468,2.98603,2.81708,18.80689,15.72800
207356,5.09520,2.75342,162.23725,14.69620
158354,4.92559,2.77315,136.77047,15.00899
165553,3.71477,2.79064,40.04929,15.29141


In [54]:
print('RMSE: %f' % (np.sqrt(mean_squared_error(xgb_check['개당낙찰업체투찰금액'], xgb_check['개당낙찰업체투찰금액_xgb_pre']))))
print('RMSLE : ', (np.sqrt(np.mean(np.square(np.log1p(xgb_check['개당낙찰업체투찰금액']) - np.log1p(xgb_check['개당낙찰업체투찰금액_xgb_pre']))))))

RMSE: 304234.460439
RMSLE :  1.1221041887467142


## RandomForest

In [55]:
rf_model = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=7525, verbose=0, warm_start=False)

In [56]:
rf_model.fit(X_train, y_train)

rf_preds = rf_model.predict(X_test)

rf_rmse = np.sqrt(mean_squared_error(y_test, rf_preds))

print('RMSE: %f' % (rf_rmse))
print('RMSLE : ', (np.sqrt(np.mean(np.square(np.log1p(y_test.iloc[:,0]) - np.log1p(rf_preds))))))

print('R2 Score : %f' % (r2_score(y_test, rf_preds)))
print('adjusted R2 Score : ', 1-(1-r2_score(y_test, rf_preds)) * (len(y_test)-1) / (len(y_test) - len(X.columns) - 1))

<ipython-input-56-6e047039d26d>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train, y_train)


RMSE: 1.126223
RMSLE :  0.30998393284602344
R2 Score : 0.948356
adjusted R2 Score :  0.9483424223301109


In [57]:
rf_check = pd.DataFrame({'log_개당낙찰업체투찰금액' : y_test.iloc[:,0],
              'log_개당낙찰업체투찰금액_rf_pre' : rf_preds})

rf_check['개당낙찰업체투찰금액'] = np.expm1(rf_check['log_개당낙찰업체투찰금액'])
rf_check['개당낙찰업체투찰금액_rf_pre'] = np.expm1(rf_check['log_개당낙찰업체투찰금액_rf_pre'])

rf_check

,log_개당낙찰업체투찰금액,log_개당낙찰업체투찰금액_rf_pre,개당낙찰업체투찰금액,개당낙찰업체투찰금액_rf_pre
243356,16.74207,16.75445,18663300.00000,18895807.61862
199304,2.88848,2.72785,16.96601,14.30001
193208,3.43420,2.73311,30.00662,14.38071
187364,1.22002,2.77110,2.38726,14.97620
270741,17.00845,16.95260,24360000.00000,23036606.54291
...,...,...,...,...
179468,2.98603,2.83442,18.80689,16.02061
207356,5.09520,2.74011,162.23725,14.48871
158354,4.92559,2.77322,136.77047,15.01012
165553,3.71477,2.76980,40.04929,14.95549


In [58]:
print('RMSE: %f' % (np.sqrt(mean_squared_error(rf_check['개당낙찰업체투찰금액'], rf_check['개당낙찰업체투찰금액_rf_pre']))))
print('RMSLE : ', (np.sqrt(np.mean(np.square(np.log1p(rf_check['개당낙찰업체투찰금액']) - np.log1p(rf_check['개당낙찰업체투찰금액_rf_pre']))))))

RMSE: 289995.629702
RMSLE :  1.1262228557133096


## final model

In [59]:
final_check = pd.DataFrame({'log_개당낙찰업체투찰금액' : y_test.iloc[:,0],
              'log_개당낙찰업체투찰금액_pre' : ((rf_preds + xgb_preds + lgbm_preds) / 3)})

final_check['개당낙찰업체투찰금액'] = np.expm1(final_check['log_개당낙찰업체투찰금액'])
final_check['개당낙찰업체투찰금액_pre'] = np.expm1(final_check['log_개당낙찰업체투찰금액_pre'])

final_check

,log_개당낙찰업체투찰금액,log_개당낙찰업체투찰금액_pre,개당낙찰업체투찰금액,개당낙찰업체투찰금액_pre
243356,16.74207,16.75807,18663300.00000,18964345.63642
199304,2.88848,2.74254,16.96601,14.52630
193208,3.43420,2.74388,30.00662,14.54723
187364,1.22002,2.76840,2.38726,14.93309
270741,17.00845,16.93798,24360000.00000,22702363.23446
...,...,...,...,...
179468,2.98603,2.81481,18.80689,15.68994
207356,5.09520,2.75375,162.23725,14.70134
158354,4.92559,2.77464,136.77047,15.03280
165553,3.71477,2.78379,40.04929,15.18016


In [61]:
print('RMSE: %f' % (np.sqrt(mean_squared_error(final_check['개당낙찰업체투찰금액'], final_check['개당낙찰업체투찰금액_pre']))))
print('RMSLE : ', (np.sqrt(np.mean(np.square(np.log1p(final_check['개당낙찰업체투찰금액']) - np.log1p(final_check['개당낙찰업체투찰금액_pre']))))))

RMSE: 279300.076296
RMSLE :  1.121134468793922


In [63]:
print('r2 score: %f' % (r2_score(final_check['개당낙찰업체투찰금액'], final_check['개당낙찰업체투찰금액_pre'])))
print('adjusted R2 Score : ', 1-(1-r2_score(final_check['개당낙찰업체투찰금액'], final_check['개당낙찰업체투찰금액_pre'])) * (len(final_check['개당낙찰업체투찰금액'])-1) / (len(final_check['개당낙찰업체투찰금액']) - len(X.columns) - 1))

r2 score: 0.998086
adjusted R2 Score :  0.9980858023633737


In [64]:
final_check[final_check['개당낙찰업체투찰금액']>=1000]

,log_개당낙찰업체투찰금액,log_개당낙찰업체투찰금액_pre,개당낙찰업체투찰금액,개당낙찰업체투찰금액_pre
243356,16.74207,16.75807,18663300.00000,18964345.63642
270741,17.00845,16.93798,24360000.00000,22702363.23446
1937,14.01250,13.85258,1217725.00000,1037767.26778
289227,15.41115,15.43591,4931460.00000,5055093.78592
98885,7.87796,7.98319,2637.48126,2930.27254
...,...,...,...,...
262817,10.42246,10.44203,33605.00000,34269.13397
30519,16.50736,16.52803,14758900.00000,15067205.83312
97865,16.55558,16.47260,15488000.00000,14254698.97494
263423,17.08361,17.08941,26261500.00000,26414256.92508


In [65]:
final_check[final_check['개당낙찰업체투찰금액']<=20]

,log_개당낙찰업체투찰금액,log_개당낙찰업체투찰금액_pre,개당낙찰업체투찰금액,개당낙찰업체투찰금액_pre
199304,2.88848,2.74254,16.96601,14.52630
187364,1.22002,2.76840,2.38726,14.93309
170490,2.35999,2.78023,9.59082,15.12276
81420,0.77035,1.45613,1.16053,3.28935
158063,1.08339,2.77464,1.95468,15.03280
...,...,...,...,...
171540,1.69577,2.79883,4.45082,15.42538
225075,2.20886,2.77790,8.10535,15.08522
196444,2.89318,2.74063,17.05061,14.49670
179468,2.98603,2.81481,18.80689,15.68994


개당 낙찰액이 두자리수 (10원 대에 가격이 형성된)를 갖는 물품에는 오차가 크게 있었지만, 개당 가격이 높은 물품에 대해서는 꽤 괜찮은 정확도를 보였다.

개당 가격으로 나타내지 않고 (금액 / 합계수량을 진행하지 않고) 모델링을 진행한다면, 결정계수의 값이 0.7정도로 낮게 나왔고, RMSE값도 더 높게 도출됨을 확인하였다.

배정예산-기초금액, 기초금액-예정가격, 예정가격-추정가격, 예정가격-배정예산, 등등.. 많은 쌍으로 실험해본 결과, 배정예산과 기초금액에서 오차가 제일 낮고 결정계수가 높게 나왔다.